In [1]:
EURI_API_KEY="euri-ac07efca88855813704bd06bf3daa4868aba1e20ed4a6243f86ad6b73e15642f"

In [2]:
from euriai.langchain import create_chat_model
from pypdf import PdfReader, PdfWriter
from typing import List, Optional
from io import BytesIO
from langchain_community.vectorstores import Chroma, Weaviate, FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List, Optional
from pathlib import Path

d:\GENAI_Bootcamp\31stAug_medichat\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PDF read function

In [3]:
def extract_text_from_pdf(file):
    reader = PdfReader(file)
    text = ' '
    for page in reader.pages:
        text = text + page.extract_text() or ''
    return text

In [4]:
# Build uploaded_files list from sample_data and extract texts into all_texts
uploaded_files = sorted(Path('sample_data').rglob('*.pdf'))
all_texts = []
for file in uploaded_files:
    try:
        text = extract_text_from_pdf(str(file))
    except Exception as e:
        print(f'Failed to extract {file}: {e}')
        text = ''
    all_texts.append(text)

# Print short preview for each extracted text
for i, (file, txt) in enumerate(zip(uploaded_files, all_texts), start=1):
    print(f'File {i}: {file} ({len(txt)} chars)')
    print(txt[:300])
    print()

File 1: sample_data\medical_history_1.pdf (688 chars)
 Medical History Report #1
Name: John Doe
Age: 45
History: John Doe, a 45-year-old male, has a history of hypertension diagnosed at age
38. He has been on antihypertensive medication since then. In 2018, he
suffered a mild myocardial infarction and underwent angioplasty. He is a
non-smoker, exercise

File 2: sample_data\medical_history_2.pdf (615 chars)
 Medical History Report #2
Name: Jane Smith
Age: 60
History: Jane Smith, a 60-year-old female, has type 2 diabetes mellitus diagnosed at
age 50. She also has osteoarthritis of the knees. She underwent left knee
replacement in 2020. She monitors her blood glucose daily and is on oral
hypoglycemics. S

File 3: sample_data\medical_history_3.pdf (535 chars)
 Medical History Report #3
Name: Michael Lee
Age: 35
History: Michael Lee, a 35-year-old male, presents with a history of asthma since
childhood. He uses inhalers as needed and has had no hospitalizations in the
past 5 years. He works

vector store

In [5]:
 # Split texts into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

In [6]:
chunks = []
for text in all_texts:
    chunks.extend(text_splitter.split_text(text))
print(chunks)

['Medical History Report #1\nName: John Doe\nAge: 45\nHistory: John Doe, a 45-year-old male, has a history of hypertension diagnosed at age\n38. He has been on antihypertensive medication since then. In 2018, he\nsuffered a mild myocardial infarction and underwent angioplasty. He is a\nnon-smoker, exercises regularly, and follows a low-sodium diet. Family history\nis significant for cardiovascular disease. Recent labs show controlled blood\npressure and normal cholesterol levels. He is compliant with medications and\nattends regular follow-ups.\nAllergies: None\nMedications: Amlodipine 5mg, Atorvastatin 20mg\nSurgeries: Angioplasty (2018)\nNotes: Patient is stable. Continue current management.', 'Medical History Report #2\nName: Jane Smith\nAge: 60\nHistory: Jane Smith, a 60-year-old female, has type 2 diabetes mellitus diagnosed at\nage 50. She also has osteoarthritis of the knees. She underwent left knee\nreplacement in 2020. She monitors her blood glucose daily and is on oral\nhypog

In [7]:
def create_faiss_index(texts: List[str]) :
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")
    # Create the FAISS index
    faiss_index = FAISS.from_texts(texts, embeddings)
    return faiss_index


def retrive_relevant_docs(vectorstore: FAISS, query: str, k: int = 3):
    return vectorstore.similarity_search(query, k=k)

In [8]:
# Create FAISS index
vectorstore = create_faiss_index(chunks)
print(vectorstore)

C:\Users\virab\AppData\Local\Temp\ipykernel_16596\3530568571.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")


chat utility function

In [9]:
API_KEY = EURI_API_KEY
MODEL = "gpt-4.1-nano"
TEMPERATURE = 0.7

In [10]:
def get_chat_model(api_key: str):
    return create_chat_model(
        api_key=api_key or API_KEY,
        model=MODEL,
        temperature=TEMPERATURE
    )

In [11]:
def ask_chat_model(chat_model, prompt: str):
    response = chat_model.invoke(prompt)
    return response.content 

In [12]:
# Initialize chat model
chat_model = get_chat_model(API_KEY)

In [13]:
chat_model

EuriaiChatModel(api_key='euri-ac07efca88855813704bd06bf3daa4868aba1e20ed4a6243f86ad6b73e15642f')

In [18]:
# Retrieve relevant documents
prompt = "John Doe"
relevant_docs = retrive_relevant_docs(vectorstore, prompt)
relevant_docs

[Document(id='c7517c50-6d20-4f44-a4bd-f3bd32dd1e18', metadata={}, page_content='Medical History Report #1\nName: John Doe\nAge: 45\nHistory: John Doe, a 45-year-old male, has a history of hypertension diagnosed at age\n38. He has been on antihypertensive medication since then. In 2018, he\nsuffered a mild myocardial infarction and underwent angioplasty. He is a\nnon-smoker, exercises regularly, and follows a low-sodium diet. Family history\nis significant for cardiovascular disease. Recent labs show controlled blood\npressure and normal cholesterol levels. He is compliant with medications and\nattends regular follow-ups.\nAllergies: None\nMedications: Amlodipine 5mg, Atorvastatin 20mg\nSurgeries: Angioplasty (2018)\nNotes: Patient is stable. Continue current management.'),
 Document(id='d1371442-4697-46b9-9618-a593dc75168b', metadata={}, page_content='Medical History Report #4\nName: Emily Clark\nAge: 28\nHistory: Emily Clark, a 28-year-old female, has hypothyroidism diagnosed at age 2

In [19]:
# Create context from relevant documents
context = "\n\n".join([doc.page_content for doc in relevant_docs])
# Create prompt with context
system_prompt = f"""You are MediChat Pro, an intelligent medical document assistant. 
                Based on the following medical documents, provide accurate and helpful answers. 
                If the information is not in the documents, clearly state that.
                While giving answers, make sure to cite the source document for reference. also,
                when you are giving an answer make sure to take help of LLM and give a full diagnosis of the problem.
                
Medical Documents:
{context}

User Question: {prompt}

Answer:"""
                
response = ask_chat_model(chat_model, system_prompt)
print(response)

Based on the provided medical history of John Doe, a 45-year-old male with a history of hypertension, previous myocardial infarction, and angioplasty, here is a comprehensive assessment:

**Diagnosis and Current Status:**
John Doe has a history of essential hypertension diagnosed at age 38, which has been well-controlled with medication (Amlodipine 5mg). His past myocardial infarction in 2018, for which he underwent angioplasty, indicates underlying coronary artery disease. His risk factors include a family history of cardiovascular disease and his prior cardiac event.

**Current Condition:**
Recent labs show controlled blood pressure and normal cholesterol levels, indicating effective management of his cardiovascular risk factors. He is compliant with medications and attends regular follow-ups, which is crucial for secondary prevention.

**Full Diagnosis:**
John's clinical profile suggests he is a patient with stable ischemic heart disease secondary to atherosclerosis, with well-manag

In [20]:
# Retrieve relevant documents
prompt = "Abhrajit Pal"
relevant_docs = retrive_relevant_docs(vectorstore, prompt)
relevant_docs

[Document(id='7457f21e-cb7c-4486-bef6-efa10cbceace', metadata={}, page_content='Medical History Report #6\nName: Priya Patel\nAge: 40\nHistory: Priya Patel, a 40-year-old female, has a history of breast cancer diagnosed at\nage 37. She underwent lumpectomy followed by chemotherapy and radiation.\nShe is currently in remission and attends oncology follow-ups every 6 months.\nNo other chronic illnesses. Family history positive for breast cancer in maternal\naunt.\nAllergies: None\nMedications: Tamoxifen 20mg\nSurgeries: Lumpectomy (2019)\nNotes: No evidence of recurrence. Continue regular surveillance.'),
 Document(id='c7517c50-6d20-4f44-a4bd-f3bd32dd1e18', metadata={}, page_content='Medical History Report #1\nName: John Doe\nAge: 45\nHistory: John Doe, a 45-year-old male, has a history of hypertension diagnosed at age\n38. He has been on antihypertensive medication since then. In 2018, he\nsuffered a mild myocardial infarction and underwent angioplasty. He is a\nnon-smoker, exercises re

In [21]:
# Create context from relevant documents
context = "\n\n".join([doc.page_content for doc in relevant_docs])
# Create prompt with context
system_prompt = f"""You are MediChat Pro, an intelligent medical document assistant. 
                Based on the following medical documents, provide accurate and helpful answers. 
                If the information is not in the documents, clearly state that.
                While giving answers, make sure to cite the source document for reference. also,
                when you are giving an answer make sure to take help of LLM and give a full diagnosis of the problem.
                
Medical Documents:
{context}

User Question: {prompt}

Answer:"""
                
response = ask_chat_model(chat_model, system_prompt)
print(response)

I'm sorry, but there is no medical information or history provided for Abhrajit Pal in the documents you shared. Therefore, I cannot offer any diagnosis or medical advice related to him. If you can provide his medical history or specific details, I will be happy to assist further.
